In [1]:
import numpy as np
import matplotlib as plt
import string
from sklearn.model_selection import train_test_split

In [2]:
archivos = [
    'Benedetti.txt',
    'Neruda.txt'
]

In [3]:
#lineas poemas
textos = []

#0 si pertenece a Benedetti y 1 si pertenece a Neruda. Una etiqueta por cada linea
etiquetas = []

In [4]:
for etiqueta, f in enumerate(archivos):
    print(f"{f} corresponde a {etiqueta}")
    
    with open(f, 'r', encoding='utf-8') as archivo:
        for line in archivo:
            #transformamos a minúsculas
            line = line.rstrip().lower()
            #eliminamos marcadores de puntuación
            if line:
                line = line.translate(str.maketrans('','',string.punctuation))
                textos.append(line)
                etiquetas.append(etiqueta)


Benedetti.txt corresponde a 0
Neruda.txt corresponde a 1


In [5]:
#Dividimos los textos y etiquetas en un conjunto de entrenamiento 
#y un conjunto de prueba

text_train, text_test, tag_train, tag_test = train_test_split(textos, 
                                                        etiquetas, 
                                                        test_size=0.1, 
                                                        random_state=42)

In [6]:
len(tag_train), len(tag_test)

(1793, 200)

In [6]:
'''
<unk> es una convención que se utiliza a menudo en el NLP para representar
palabras desconocidas o fuera del vocabulario. En este caso, se está asignando
el índice 0 a esta palabra especial.
'''

indice = 1
indicepalabras = {'<unk>':0}

'''
Hacemos esto porque puede que en el conjunto de prueba haya palabras
que no hayan sido entrenadas en el modelo (porque no estaban en el conjunto
de entrenamiento)
'''

'\nHacemos esto porque puede que en el conjunto de prueba haya palabras\nque no hayan sido entrenadas en el modelo (porque no estaban en el conjunto\nde entrenamiento)\n'

In [7]:
#Construcción del diccionario de las palabras a índices:
for texto in text_train:
    tokens = texto.split()
    for token in tokens:
        if token not in indicepalabras:
            indicepalabras[token] = indice
            indice += 1
            

In [14]:
#cada palabra tiene un índice: diccionario de palabras únicas
indicepalabras

{'<unk>': 0,
 'sin': 1,
 'tanta': 2,
 'cortedad': 3,
 'los': 4,
 'de': 5,
 'ella': 6,
 'hemos': 7,
 'visto': 8,
 'arder': 9,
 'tantas': 10,
 'veces': 11,
 'el': 12,
 'lucero': 13,
 'besándonos': 14,
 'ojos': 15,
 'su': 16,
 'desnudez': 17,
 'indemne': 18,
 'tan': 19,
 'carnal': 20,
 'algo': 21,
 'aprendido': 22,
 'a': 23,
 'pedacitos': 24,
 'y': 25,
 'pulsaciones': 26,
 'millones': 27,
 'huéspedes': 28,
 'que': 29,
 'se': 30,
 'aburren': 31,
 'pero': 32,
 'igual': 33,
 'me': 34,
 'sorprendo': 35,
 'diez': 36,
 'no': 37,
 'dejar': 38,
 'la': 39,
 'paciencia': 40,
 'ceda': 41,
 'cuando': 42,
 'llegaron': 43,
 'casa': 44,
 'pretextos': 45,
 'ni': 46,
 'del': 47,
 'tiempo': 48,
 'en': 49,
 'torno': 50,
 'mí': 51,
 'estoy': 52,
 'viendo': 53,
 'tu': 54,
 'cintura': 55,
 'niebla': 56,
 'ha': 57,
 'venido': 58,
 'dormirse': 59,
 'vientre': 60,
 'una': 61,
 'mariposa': 62,
 'sombra': 63,
 'ríen': 64,
 'comen': 65,
 'pesar': 66,
 'veta': 67,
 'dirás': 68,
 'pesados': 69,
 'como': 70,
 'juicios'

In [8]:
#Convertimos los datos en enteros:

text_train_int = []
text_test_int = []

for linea in text_train:
    tokens = linea.split()
    line_as_int = [indicepalabras[token] for token in tokens]
    text_train_int.append(line_as_int)

In [9]:
for linea in text_test:
    tokens = linea.split()
    line_as_int = [indicepalabras.get(token, 0) for token in tokens]
    text_test_int.append(line_as_int)

In [10]:
#Modelo de Markov: matrices de transición y 
#probabilidades iniciales de las palarbas
#Matrices con puros unos para hacer el suavizado +1

V = len(indicepalabras)

#matriz de transición para textos de Benedetti
A0 = np.ones((V,V))

#probabilidades iniciales para textos de Benedetti
pi0 = np.ones(V)

#matriz de transición para textos de Neruda
A1 = np.ones((V,V))

#probabilidades iniciales para textos de Neruda
pi1 = np.ones(V)


In [11]:
#Calculamos probabilidades

def compute_counts(text_as_int, A, pi):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            #si estoy en la primera palabra de la secuencia:
            if last_idx is None:
                pi[idx] += 1
            else:
                A[last_idx, idx] += 1
            last_idx = idx

compute_counts([t for t, y in zip(text_train_int, tag_train) if y==0], A0, pi0)
compute_counts([t for t, y in zip(text_train_int, tag_train) if y==1], A1, pi1)

#t representa cada linea de texto en text_train_int
#y representa cada tag en tag_train
#zip crea una tupla con cada linea y cada tag, y como son muchas tenemos
#una lista de tuplas
#el filtro y==0 o y==1 se encarga de ver que solo se incluyan las tuplas
#con las líneas correspondientes a Benedetti o Neruda

#t for t, y in zip(...) if ... --> "dame cada elemento t de cada tupla "t,y" 
#de [...] si cumple tal condición"

In [12]:
#Normalizamos A y pi para que sean matrices de probabilidad válidas

A0 /= A0.sum(axis=1, keepdims=True) 
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

#axis indica que se realiza la suma sobre el eje y (las columnas)
#keepdims indica que se deben mantener las dimensiones de la matriz


In [13]:
#Usamos un espacio logarítmico para que los nros no sean tan chicos 
#y no genere problemas de desbordamiento

logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)


In [14]:
#Cuenta las etiquetas de cada clase en tag_train
count0 = sum(y == 0 for y in tag_train)
count1 = sum(y == 1 for y in tag_train)

total = len(tag_train) #cant total de ejemplos de entrenamiento

p0 = count0/total #Probabilidad a priori de clase 0
p1 = count1/total #Probabilidad a priori de clase 1

#Logaritmos de probabilidades a priori
logp0 = np.log(p0)
logp1 = np.log(p1)

p0, p1

(0.7479085331846068, 0.2520914668153932)

In [15]:
#Construcción del clasificador:

class Classifier:
    def __init__(self, logAs, logpis, logpriors): #acá le pasa listas?
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors) #nro de clases

    def _compute_log_likelihood(self, input_, class_): 
        #input_ --> representa las palabras del texto en enteros (las lineas)
        #class_ --> son las clases definidas por K (en este caso 0 y 1)
        

        logA = self.logAs[class_]
        logpi = self.logpis[class_]

        last_idx = None

        logprob = 0

        for idx in input_:
            if last_idx is None:
                #si es el primer token de la secuencia
                logprob += logpi[idx]
            else:
                #Calcula la probabilidad de transición de la palabra anterior a la actual
                logprob += logA[last_idx, idx]

            #Actualiza last_idx para la próxima iteración
            last_idx = idx

        return logprob

    def predict(self, inputs): 
        #el input en este caso van a ser los textos en enteros
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            #Calcula los logaritmos de las probabilidades 
            #posteriores para cada clase
            posteriores = [self._compute_log_likelihood(input_, c) + 
                           self.logpriors[c] for c in range(self.K)]
                        #se manda c dos veces (una para 0 y otra para 1), y esp
                        #se decide con el for c in range(self.K), es decir
                        #para cada c dentro de ese rango
                        
                        #usa self.logpriors o sea cada probabilidad logarítmica
                        #de c, o sea de K, o sea cada uno de los dos nros
                        #almacenados en logpriors (que es una lista), 
                        #o sea las probs de que sea 0 o 1

                        #se suman las probabilidades para cada linea 
                        #con las probabilidades a priori

            #Elige la clase con mayor prob posterior como la predicción
            pred = np.argmax(posteriores)
            predictions[i] = pred

        return predictions

In [16]:
#Cada arreglo debe estar en orden ya que se asume que 
#las clases indexan estas listas:

clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])


In [17]:
Ptrain = clf.predict(text_train_int)
print(f"Train acc: {np.mean(Ptrain == tag_train)}")

Train acc: 0.992749581706637


In [18]:
Ptest = clf.predict(text_test_int)
print(f"Test acc: {np.mean(Ptest == tag_test)}")

Test acc: 0.83
